<a href="https://colab.research.google.com/github/zacharylazzara/tent-detection/blob/main/tent_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
if [ ! -d "./unet" ]; then
 git clone https://github.com/tofighi/sarpol-zahab-tents.git
 git clone https://github.com/zhixuhao/unet.git
fi

# For some reason line 55 of model.py needs to be replaced by model = Model(inputs, conv10)
sed -i '55s/.*/    model = Model(inputs, conv10)/' unet/model.py

mkdir unet/data/tents
cp -r sarpol-zahab-tents/data unet/data/tents/train

cd unet/data/tents/train

mv images image
mv labels label

cd image
for file in .
do
  rename 's/.{7}(.*)/$1/' *
done

cd ../label
for file in .
do
  rename 's/.{7}(.*)/$1/' *
done

cd ..
mkdir aug # TODO: we need to augment the data then load it as our training data


cd

Cloning into 'sarpol-zahab-tents'...
Cloning into 'unet'...


In [2]:
%cd unet/
# data_root = "data/membrane/"
data_root = "data/tents/"

from model import *
from data import *

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2, data_root + 'train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])


imgs_train,imgs_mask_train = geneTrainNpy(data_root + "train/aug/",data_root + "train/aug/")
model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])


testGene = testGenerator(data_root + "test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult(data_root + "test",results)

/content/unet


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 256 images belonging to 1 classes.
Found 256 images belonging to 1 classes.
Epoch 1/5
2000/2000 [==============================] - 374s 160ms/step - loss: 0.6692 - accuracy: 0.9951

Epoch 00001: loss improved from inf to 0.64575, saving model to unet_membrane.hdf5
Epoch 2/5
2000/2000 [==============================] - 327s 164ms/step - loss: 0.5794 - accuracy: 0.9975

Epoch 00002: loss improved from 0.64575 to 0.55897, saving model to unet_membrane.hdf5
Epoch 3/5
2000/2000 [==============================] - 328s 164ms/step - loss: 0.5006 - accuracy: 0.9975

Epoch 00003: loss improved from 0.55897 to 0.48259, saving model to unet_membrane.hdf5
Epoch 4/5
2000/2000 [==============================] - 328s 164ms/step - loss: 0.4312 - accuracy: 0.9975

Epoch 00004: loss improved from 0.48259 to 0.41525, saving model to unet_membrane.hdf5
Epoch 5/5
2000/2000 [==============================] - 328s 164ms/step - loss: 0.3701 - accuracy: 0.9975

Epoch 00005: loss improved from 0.41525 to 0

ValueError: ignored